In [1]:
import pyodbc
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles.numbers import FORMAT_NUMBER_COMMA_SEPARATED1

def connect_to_sql_server(server, database, user, password):
    """Connects to a SQL Server database and returns a connection object."""
    conn_str = (
        r'DRIVER={SQL Server};'
        r'SERVER=' + server + ';'
        r'DATABASE=' + database + ';'
        r'UID=' + user + ';'
        r'PWD=' + password + ';'
    )
    conn = pyodbc.connect(conn_str)
    return conn

def get_project_totals(conn):
    """Gets the total cash out for each project."""
    query = """
        SELECT project_name, SUM(equiv) AS total_cash_out
        FROM [omar.rme1].[dbo].[ap_check]
        GROUP BY project_name
    """
    df = pd.read_sql(query, conn)
    return df

def create_report(df):
    """Creates a well-formatted Excel report."""
    wb = Workbook()
    ws = wb.active
    ws.title = "Project Cash Out Report"

    # Adding header
    ws.append(["Project Name", "Total Cash Out (in equiv)"])

    # Adding the data
    for row in dataframe_to_rows(df, index=False, header=False):
        ws.append(row)

    # Apply formatting
    header_font = Font(bold=True)
    for cell in ws["1:1"]:  # First row, the header
        cell.font = header_font
        cell.alignment = Alignment(horizontal='center')

    # Format the 'Total Cash Out' column
    for cell in ws["B"]:
        if cell.row != 1:  # Skip the header
            cell.number_format = FORMAT_NUMBER_COMMA_SEPARATED1

    # Adjust column widths
    ws.column_dimensions["A"].width = 40  # Project Name
    ws.column_dimensions["B"].width = 20  # Total Cash Out

    # Save the workbook
    report_filename = "Project_Cash_Out_Report.xlsx"
    wb.save(report_filename)
    print(f"Created {report_filename}")

if __name__ == "__main__":
    server = '10.10.11.241'
    database = 'omar.rme1'
    user = 'yasser'
    password = 'yasser123'

    try:
        conn = connect_to_sql_server(server, database, user, password)
        df = get_project_totals(conn)
        create_report(df)

    except Exception as e:
        print("Error:", e)
    finally:
        if conn:
            conn.close()


C:\Users\Omar Essam\AppData\Local\Temp\ipykernel_16984\2040679261.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Created Project_Cash_Out_Report.xlsx
